In [15]:
import ee
import geemap
import pprint
from tabulate import tabulate

ee.Authenticate
ee.Initialize(project='toribiodiego-ece471')

# Homework 2
- ECE471: Fundamentals of Remote Sensing and Earth Observation
- Spring 2025
- Instructor: Krishna Karra
- Contact: krishna.karra@cooper.edu (Teams chat is quickest)
- Assigned: Wednesday, February 26, 2025
- Due: Tuesday, March 18, 2025 (by midnight EST)

## Problem Motivation

The United States Department of Agriculture (USDA), in conjunction with the United States Geological Survey (USGS), produces the Cropland Data Layer (CDL), an annual landcover map of crop types across the Continental United States at 30-meter resolution. This map, which incorporates Landsat and Sentinel imagery from across the year, represents some of the highest quality, rigorously validated land cover classification that can be found anywhere.

A year's annual map is usually released a few months into the next year (e.g. the 2020 CDL is released towards the end of the first quarter of 2021). As a geospatial consultant, your task is to investigate whether certain crop types could be predicted *in-season* (e.g. without using a full year's worth of imagery).

The CDL is used within government and industry for crop health monitoring, crop area estimation, agricultural policy development, environmental monitoring, yield forecasting and more. In-season prediction of crop types would be vastly useful for all of these applications.

For this pilot study, the focus will be on Iowa and Illinois.

## Problem Overview

The overall workflow will be:

1. Analyze cropland data to identify the 5 most prevalent crops grown in Iowa and Illinois.
2. Explore and visualize the seasonal patterns of those 5 crops using vegetation index time series analysis.
3. Develop a crop identification model that takes satellite imagery as input and predicts crop type at the pixel level.
4. Assess model performance using validation data and metrics.
5. Apply the model to full counties to produce crop type maps and compare against ground truth data.

The core data sources will be:

1. Cropland Data Layer (CDL) for crop type ground truth labels
2. Landsat satellite imagery for model inputs
3. Google Earth Engine (GEE) platform for analysis and modeling

This assignment will give you real, concrete experience with:

- Working with large-scale geospatial data
- Time series analysis of vegetation indices
- Developing and validating a remote sensing model
- Creating meaningful data visualizations and maps

Your modeling approach and additional data sources are flexible. The key deliverables are: ranked crop list, crop seasonal analysis, trained crop identification model, model evaluations, and final county-level crop maps.

## Task 1: Identify Most Prevalent Crops

###*Goal: Plan to have made significant progress on Task 1 (or completed it) by March 5.*

The goal of this task is to create a ranked list of the most commonly grown crops across Iowa and Illinois based on acreage data from the 2019 Cropland Data Layer (CDL).

Specifically, you should:

1. Load the 2019 CDL raster data for the 2 states from Google Earth Engine.
2. Write a script to calculate the total acreage of each unique crop type in the CDL legend across the 2 states.
3. Rank the crops based on total acreage from most to least.
4. Output the list as a CSV file with two columns: Crop Name and Total Acres. Include the top 20 crops. Include the top 20 crops.
5. From this list, select the top 5 most prevalent crops. These will be the focus for the rest of the analysis.

Things to keep in mind:

1. Analysis must be done programmatically using GEE.
2. Output must be a clean CSV file with the columns specified.
3. Make sure to handle any data filtering or preprocessing steps like reprojections.
4. Document your methodology and thought process

### 1.1 -

#### 1.1.1 - Fixed Test Region (Central Iowa)

In [49]:
# Create an interactive map centered over Iowa
Map = geemap.Map(center=[42.0, -93.5], zoom=7)

In [76]:
# Define a small test region using a rectangular geometry.
test_region = ee.Geometry.Rectangle([-93.75, 41.6, -93.6, 41.7])

# Add the small test region to the map with a red border
Map.addLayer(test_region, {'color': 'red'}, 'Test Region')

#### 1.1.2 - Counties

In [52]:
# Load the counties FeatureCollection from TIGER/2018
counties = ee.FeatureCollection("TIGER/2018/Counties")

# Filter to include only counties in Iowa.
# Iowa's state FIPS code is "19".
iowa_counties = counties.filter(ee.Filter.eq('STATEFP', '19'))

# Add the Iowa counties to the map with a blue outline
Map.addLayer(iowa_counties.style(**{'color': 'blue'}), {}, 'Iowa Counties')

In [62]:
# Display the map
Map


Map(bottom=3539.0, center=[35.209721645221386, -95.03173828125001], controls=(WidgetControl(options=['position…

In [74]:
# load the 2019 USDA Cropland Data Layer image.
cdl_2019 = ee.Image('USDA/NASS/CDL/2019')

# create a reducer that sums pixel areas and groups by the 'cropland' band (crop code at index 1).
reducer = ee.Reducer.sum().group(groupField=1, groupName='crop_code')

# calculate pixel area in acres (each pixel is 30m x 30m, and 1 m² ≈ 0.000247105 acres).
area_image = ee.Image.pixelArea().multiply(0.000247105)

stats_small_region = area_image.addBands(cdl_2019.select('cropland')).reduceRegion(
    reducer=reducer,          # The reducer to apply: sums pixel areas grouped by crop code.
    geometry=test_region,       # The geographic region over which to perform the reduction.
    scale=30,                   # The pixel resolution in meters; here, each pixel is 30m x 30m.
    maxPixels=1e13              # Maximum number of pixels allowed in the calculation to avoid errors.
)


# extract the grouped acreage results by crop code.
grouped_small_region = stats_small_region.get('groups')

In [89]:
# Retrieve the 'cropland_class_names' and 'cropland_class_values' properties from the 2019 CDL image.
crop_class_names = cdl_2019.get('cropland_class_names').getInfo()
crop_class_values = cdl_2019.get('cropland_class_values').getInfo()

# Convert the crop class names to a list by splitting the comma-separated string.
crop_class_names = crop_class_names.split(',')

# Convert the crop class values to a list of integers by splitting the string and converting each element.
crop_class_values = [int(val) for val in crop_class_values.split(',')]

# Create a dictionary mapping crop codes (integers) to crop names.
crop_legend = dict(zip(crop_class_values, crop_class_names))

print("Crop Legend:")
print(crop_legend)

Crop Legend:
{0: 'Background', 1: 'Corn', 2: 'Cotton', 3: 'Rice', 4: 'Sorghum', 5: 'Soybeans', 6: 'Sunflower', 10: 'Peanuts', 11: 'Tobacco', 12: 'Sweet Corn', 13: 'Pop or Orn Corn', 14: 'Mint', 21: 'Barley', 22: 'Durum Wheat', 23: 'Spring Wheat', 24: 'Winter Wheat', 25: 'Other Small Grains', 26: 'Dbl Crop WinWht/Soybeans', 27: 'Rye', 28: 'Oats', 29: 'Millet', 30: 'Speltz', 31: 'Canola', 32: 'Flaxseed', 33: 'Safflower', 34: 'Rape Seed', 35: 'Mustard', 36: 'Alfalfa', 37: 'Other Hay/Non Alfalfa', 38: 'Camelina', 39: 'Buckwheat', 41: 'Sugarbeets', 42: 'Dry Beans', 43: 'Potatoes', 44: 'Other Crops', 45: 'Sugarcane', 46: 'Sweet Potatoes', 47: 'Misc Vegs & Fruits', 48: 'Watermelons', 49: 'Onions', 50: 'Cucumbers', 51: 'Chick Peas', 52: 'Lentils', 53: 'Peas', 54: 'Tomatoes', 55: 'Caneberries', 56: 'Hops', 57: 'Herbs', 58: 'Clover/Wildflowers', 59: 'Sod/Grass Seed', 60: 'Switchgrass', 61: 'Fallow/Idle Cropland', 63: 'Forest', 64: 'Shrubland', 65: 'Barren', 66: 'Cherries', 67: 'Peaches', 68: 'Ap

In [91]:
# retrieve the list of county features (this uses getInfo() so it runs on the client).
iowa_counties_list = iowa_counties.getInfo()['features']

# conversion factor from square meters to acres.
sqm_to_acres = 0.000247105

county_land_area = {}
for feature in iowa_counties_list:
    props = feature['properties']
    county_name = props['NAME']
    # Convert ALAND (square meters) to acres and round to two decimal places.
    land_area_acres = round(props['ALAND'] * sqm_to_acres, 2)
    county_land_area[county_name] = land_area_acres

# sorted list of (county_name, land_area) tuples by acreage.
sorted_counties = sorted(county_land_area.items(), key=lambda x: x[1])
smallest_county = sorted_counties[0]
largest_county = sorted_counties[-1]

# for median, if the list length is even, we choose the lower median.
median_index = len(sorted_counties) // 2
median_county = sorted_counties[median_index]

print("\nSmallest county:", smallest_county)
print("Median county:", median_county)
print("Largest county:", largest_county)

# helper function to retrieve the ee.Geometry of a county given its name
def get_county_geometry(county_name):
    # Filter the FeatureCollection to the county with the given name.
    county_feature = iowa_counties.filter(ee.Filter.eq('NAME', county_name)).first()
    return county_feature.geometry()

# retrieve the ee.Geometry for each county.
smallest_geom = get_county_geometry(smallest_county[0])
median_geom = get_county_geometry(median_county[0])
largest_geom = get_county_geometry(largest_county[0])


Smallest county: ('Dickinson', 243532.42)
Median county: ('Greene', 364554.8)
Largest county: ('Kossuth', 622546.9)


In [60]:
def compute_crop_acreage_table(region, crop_legend=crop_legend):
    """
    Computes the total acreage by crop code for the given region,
    maps crop codes to crop names, ranks them by acreage,
    rounds the acreage values to two decimal places,
    and returns a formatted table string.

    Parameters:
      region: An ee.Geometry defining the area of interest.
      crop_legend: A dictionary mapping crop codes (int) to crop names (str).

    Returns:
      A formatted string containing the table with columns:
      Rank, Crop Code, Crop Name, Total Acres.
    """
    # Load the 2019 CDL image.
    cdl_2019 = ee.Image('USDA/NASS/CDL/2019')

    # Create a reducer that sums the area (weighted input)
    # and groups by the crop code (grouping input, which is in the second band, index=1).
    reducer = ee.Reducer.sum().group(
        groupField=1,  # crop code is in the second band (index 1)
        groupName='crop_code'
    )

    # Each pixel is 30 m x 30 m (900 m^2).
    # Create an area image (in acres).
    area_image = ee.Image.pixelArea().multiply(0.000247105)

    # Add the 'cropland' band from the CDL image as the second band.
    combined_image = area_image.addBands(cdl_2019.select('cropland'))

    # Reduce the image over the provided region.
    stats = combined_image.reduceRegion(
        reducer=reducer,
        geometry=region,
        scale=30,
        maxPixels=1e13
    )

    # Retrieve the computed list as a Python object.
    groups = stats.get('groups').getInfo()

    # Prepare table data, adding crop names and rounding acreage.
    table_data = []
    for entry in groups:
        crop_code = entry['crop_code']
        acreage = round(entry['sum'], 2)  # Round to two decimal places
        crop_name = crop_legend.get(crop_code, "Unknown")
        table_data.append({
            "Crop Code": crop_code,
            "Crop Name": crop_name,
            "Total Acres": acreage
        })

    # Sort table data by Total Acres in descending order.
    table_data_sorted = sorted(table_data, key=lambda x: x["Total Acres"], reverse=True)

    # Add the Rank column.
    for idx, row in enumerate(table_data_sorted, start=1):
        row["Rank"] = idx

    # Rearrange columns so that Rank comes first.
    table_data_ranked = []
    for row in table_data_sorted:
        table_data_ranked.append({
            "Rank": row["Rank"],
            "Crop Code": row["Crop Code"],
            "Crop Name": row["Crop Name"],
            "Total Acres": row["Total Acres"]
        })

    # Create a formatted table using tabulate.
    table_str = tabulate(table_data_ranked, headers="keys", tablefmt="grid")
    return table_str

In [75]:
test_region_table = compute_crop_acreage_table(test_region)
print(test_region_table)

+--------+-------------+--------------------------+---------------+
|   Rank |   Crop Code | Crop Name                |   Total Acres |
+========+=============+==========================+===============+
|      1 |         122 | Developed/Low Intensity  |      11223.6  |
+--------+-------------+--------------------------+---------------+
|      2 |         121 | Developed/Open Space     |       6809.91 |
+--------+-------------+--------------------------+---------------+
|      3 |         123 | Developed/Med Intensity  |       3931.37 |
+--------+-------------+--------------------------+---------------+
|      4 |         190 | Woody Wetlands           |       2124.43 |
+--------+-------------+--------------------------+---------------+
|      5 |         176 | Grass/Pasture            |       2031.98 |
+--------+-------------+--------------------------+---------------+
|      6 |         141 | Deciduous Forest         |       1794.35 |
+--------+-------------+------------------------

In [72]:
smallest_table = compute_crop_acreage_table(smallest_geom, crop_legend)
print("\nCrop acreage table for smallest county:\n", smallest_table)


Crop acreage table for smallest county:
 +--------+-------------+--------------------------+---------------+
|   Rank |   Crop Code | Crop Name                |   Total Acres |
+========+=============+==========================+===============+
|      1 |           1 | Corn                     |      94176.2  |
+--------+-------------+--------------------------+---------------+
|      2 |           5 | Soybeans                 |      76273.1  |
+--------+-------------+--------------------------+---------------+
|      3 |         176 | Grass/Pasture            |      25505.1  |
+--------+-------------+--------------------------+---------------+
|      4 |         195 | Herbaceous Wetlands      |      17314.4  |
+--------+-------------+--------------------------+---------------+
|      5 |         111 | Open Water               |      15564    |
+--------+-------------+--------------------------+---------------+
|      6 |         121 | Developed/Open Space     |      10184.7  |
+-----

In [70]:
median_table = compute_crop_acreage_table(median_geom, crop_legend)
print("\nCrop acreage table for median county:\n", median_table)


Crop acreage table for median county:
 +--------+-------------+--------------------------+---------------+
|   Rank |   Crop Code | Crop Name                |   Total Acres |
+========+=============+==========================+===============+
|      1 |           1 | Corn                     |     177537    |
+--------+-------------+--------------------------+---------------+
|      2 |           5 | Soybeans                 |     120595    |
+--------+-------------+--------------------------+---------------+
|      3 |         176 | Grass/Pasture            |      23397.8  |
+--------+-------------+--------------------------+---------------+
|      4 |         121 | Developed/Open Space     |      13267.1  |
+--------+-------------+--------------------------+---------------+
|      5 |         141 | Deciduous Forest         |      11483.9  |
+--------+-------------+--------------------------+---------------+
|      6 |         190 | Woody Wetlands           |       4750.91 |
+-------

In [71]:
largest_table = compute_crop_acreage_table(largest_geom, crop_legend)
print("\nCrop acreage table for largest county:\n", largest_table)


Crop acreage table for largest county:
 +--------+-------------+--------------------------+---------------+
|   Rank |   Crop Code | Crop Name                |   Total Acres |
+========+=============+==========================+===============+
|      1 |           1 | Corn                     |     317292    |
+--------+-------------+--------------------------+---------------+
|      2 |           5 | Soybeans                 |     205863    |
+--------+-------------+--------------------------+---------------+
|      3 |         176 | Grass/Pasture            |      32508.4  |
+--------+-------------+--------------------------+---------------+
|      4 |         121 | Developed/Open Space     |      21438.5  |
+--------+-------------+--------------------------+---------------+
|      5 |         195 | Herbaceous Wetlands      |      14077.1  |
+--------+-------------+--------------------------+---------------+
|      6 |         190 | Woody Wetlands           |       6557.63 |
+------

In [92]:
# load the US States polygons from the TIGER dataset.
states = ee.FeatureCollection('TIGER/2018/States')

# filter for Iowa (IA) and Illinois (IL) using their postal abbreviations.
target_states = states.filter(ee.Filter.inList('STUSPS', ['IA', 'IL']))

# combine the state polygons into a single geometry.
states_geometry = target_states.geometry()

# compute and print the crop acreage table for the combined Iowa + Illinois region.
print("Crop acreage table for Iowa + Illinois in 2019 CDL:")
print(compute_crop_acreage_table(states_geometry, crop_legend))

Crop acreage table for Iowa + Illinois in 2019 CDL:
+--------+-------------+--------------------------+------------------+
|   Rank |   Crop Code | Crop Name                |      Total Acres |
+========+=============+==========================+==================+
|      1 |           1 | Corn                     |      2.40445e+07 |
+--------+-------------+--------------------------+------------------+
|      2 |           5 | Soybeans                 |      1.88606e+07 |
+--------+-------------+--------------------------+------------------+
|      3 |         141 | Deciduous Forest         |      7.93551e+06 |
+--------+-------------+--------------------------+------------------+
|      4 |         176 | Grass/Pasture            |      7.37887e+06 |
+--------+-------------+--------------------------+------------------+
|      5 |         121 | Developed/Open Space     |      2.69074e+06 |
+--------+-------------+--------------------------+------------------+
|      6 |         122 | 

## Task 2: Crop Seasonal Analysis

###*Goal: Plan to have made significant progress by March 5*

Goal: Analyze the seasonal patterns of the top 5 crops using vegetation index time series.

Overview:

1. Sample pixels growing for each crop across the two states
2. Compute NDVI time series for each crop sample
3. Generate visualizations of the NDVI curves
4. Perform additional analysis on crop emergence, harvest dates, etc (optional)

This will reveal unique temporal signatures of the crops and inform model development.

### Task 2.1: Sample Crop Pixel Locations

- Write a script to randomly sample at least 250 pixel locations for each of the 5 crops.
- Spread the sampling across the 2 states to capture geographical diversity.
- Output the sampled points as a FeatureCollection in GEE.
- Visualize the points to ensure they look as expected.
- Document your sampling methodology (e.g. spatial filters, randomization, etc.)
- The sampling script should be modular to allow increasing the sampling size.

### Task 2.2: Calculate NDVI Time Series

- For each crop, use the sampled points to extract NDVI values over a full year.
- Use all available Landsat surface reflectance imagery from 2019.
- Handle clouds/shadows by filtering out low quality NDVI values.
- Calculate the mean, 1-sigma, and 2-sigma NDVI curves for each crop across the time series.
- Plot the NDVI curves with date on x-axis and NDVI on y-axis.
- Include the crop name, number of sample points, and time range in the plot.
- Document your approach for filtering, aggregating, and plotting the time series data.

Key requirements:

- Use GEE reducers to extract NDVI values per pixel.
- Make sure to mask clouds and filter low quality NDVI.
- Plot mean, 1-sigma, and 2-sigma to show variability across many samples
- Output clean, labeled, and informative plots.

### Task 2.3: Additional Analysis (optional but recommended)

Perform any additional analysis on the crops that you find interesting or useful for modeling. This is an open-ended exploration task to dive deeper into the data. Document your methodology and insights gained from any additional analysis. Visualizations and quantitative metrics are highly encouraged. This is your opportunity to freely explore the data and extract patterns that could help with crop modeling in Task 3. Be creative and have fun with the analysis!

*Any additional analysis that provides unique insights or utility for crop modeling may receive extra credit.* The depth and uniqueness of the exploration will determine the amount of extra credit awarded.

Some examples of analysis that could warrant extra credit:
- Extracting crop rotational patterns across multiple years
- Quantifying optimal temporal windows for distinguishing between crops (e.g. dormancy, emergence, harvest)
- Analyzing the utility of additional indices on top of NDVI
- Incorporating time series from sensors across the Landsat mission (e.g. L5/L7/L8/L9)

## Task 3: Crop Identification Modeling

Goal: Develop and validate a crop classification model using satellite imagery as input.

Overview:

1. Use the sampling script from Task 2 to generate training data for developing a classification model.
2. The model should predict the 5 major crops identified in Task 1, plus a 6th "other" class.
3. The input features should include data (raw and/or derived) from Landsat surface reflectance imagery.
5. Validate model performance by calculating precision, recall, F1 score metrics on a held-out test set.
6. Apply the model to full counties in Iowa and Illinois to create crop classification maps.
7. Visually compare the model output maps to the Cropland Data Layer for analysis.

### Task 3.1: Model Development

Develop a classification model to predict crop type using satellite imagery and temporal vegetation data as input features. The model should be trained on the sampled points from Task 2 and output predictions for the 5 major crops plus a "other" class.

Key Requirements:

- Use sampled crop points (or sampling script) from Task 2 as training data.
- You may want to include more samples for the model than what was created in Task 2.
- Input features must include, but are not limited to, Landsat surface reflectance imagery from the growing season.
- Incorporate temporal vegetation indices like NDVI time series.
- Output predictions for the 5 major crops identified in Task 1 plus an "other" class. This "other" class could be expanded into multiple classes if deemed useful.
- Document modeling methodology including:
  - Model type, input features, training approach
  - Any data preprocessing/augmentation
  - Tools and libraries used
  - Modeling can be fully in GEE or combine external tools like Scikit-learn, Tensorflow, etc.

Remember that ideally, we are attempting to build a model that does NOT require data across the entire year. Put another way, the less data our model needs about the growing season to make a good prediction about the crop type, the better!

If this proves challenging, start by using an entire year's worth of data for your model (like CDL). Achieving good performance with your classifier with a full year's worth of data will earn you full credit, and extra credit will be given to in-season prediction.

### Task 3.2: Model Testing

Validate the performance of the developed crop classification model using accuracy metrics calculated on a held-out test set. This test set can be created with the same sampling strategy from before, but should be created indepdently of the training and validation data (e.g. not be seen by the model at all before testing).

Key Requirements:

- Reserve 10-20% of the training data to hold out for testing.
- Calculate precision, recall and F1 score metrics on the test set.
- Generate a confusion matrix showing per-class accuracies and errors.
- Analyze the results - where does the model perform well or poorly?
- The validation script must enable testing on new unseen data.

Interpret what the results imply about the real-world viability of the model. It's okay if you don't think your model is anywhere nearly as good as CDL is. Identify areas for improvement and potential next steps to further improve performance. Finally, document insights gained from the validation process and analysis of the results.

### Task 3.3: Model Deployment

We're at the end, now it's time to see how our model does over an entire county! Apply the developed crop classification model to full counties to create prediction maps and visually compare against ground truth data.

Key Requirements:

- Script should take any county as an input and produce an output classification.
- Specifically generate output classification results for two counties: (1) Crawford County, Iowa (2) Grant County, Wisconsin. (NOTE we did not train at all in Wisconsin!)
- Run the model over all the pixels of each county.
- Output crop classification maps with predictions at 30 meter resolution.
- Visually compare the model outputs to the Cropland Data Layer (CDL).
- Document any data exports or external processing required.

If your model was built entirely in GEE (recommended), deployment should be straightforward. If your model however was built outside of GEE and you had to export lots of data to run your mode, you may need to ingest results back into GEE (if needed) for visualization and comparison.

Analyze similarities and differences between your model output and the CDL. Do they line up at all? Is your approach viable for real-world deployment? How do the results differ when looked at a much larger geographic scale compared to the test set?

## Tips
- Start small and iterate! Begin by testing workflows on small counties before expanding to the full two states. This will help identify any issues early.
- Leverage the vast data in GEE! The Cropland Data Layer and Landsat imagery on GEE provide a rich dataset. But feel free to incorporate any other relevant data sources available.
- Plan exporting data from GEE! For large regional analysis, you may need to batch export preprocessed data to your local environment for modeling.
- Store off computationally heavy intermediate results! you may want to store intermediate FeatureCollections and Images as GEE assets such that you do not have to dynamically re-create them every time.
- Documentation is key! Thoroughly document your methodology, thought process, experiments, and results. This transparency is important for full credit.
- Visualizations tell a story! Use insightful visuals like time series plots and comparison maps to showcase your findings.
- There are many valid approaches! This assignment is open-ended by design. There are many "correct" ways to analyze the data and build models.
- Ask questions early! Please reach out if you need any guidance or get stuck. I'm happy to provide clarification and nudges in the right direction.